In [1]:
shows = ["Dear Evan Hansen", "Hadestown", "Phantom of the Opera", "Annie", "Ain't To Proud", "Mean Girls", "Secret Garden",
"Hairspray", "Six", "Jersey Boys", "Heathers", "Les Miserables", "Moulin Rouge", "Tina", "Mamma Mia", "Six", "MJ: The Musical", "POTUS",
"Chicago", "The Book of Mormon", "The Notebook", "Sweeney Todd", "Funny Girl", "Come From Away", "Newsies", "Wicked", "Company", "Kimberly Akimbo",
"Waitress", "Cabaret", "Fiddler on the Roof", "The Outsiders", "Back to the Future", "La Cage aux Folles", "Waiting for Godot"]

In [20]:
import pandas as pd 

df = pd.read_json('pocket_patron/scrape2.json')

In [14]:
df['image_urls'] = df['image_urls'].apply(lambda x: x[0])

In [21]:
def get_image_filename(x):
    try:
        return x[0]["path"][:-4] + ".png"
    except IndexError:
        return None

df['image_filename'] = df['images'].apply(lambda x: get_image_filename(x))

In [11]:
s = "string.png"
s[:-4]

'string'

In [22]:
s = "Sign up for announcements and exclusive discounts on tickets to your favorite shows! Blocking belongson the stage,not on websites.Our website is made possible bydisplaying online advertisements to our visitors.Please consider supporting us bywhitelisting playbill.com with your ad blocker.Thank you!"
df["description"] = df["description"].str.replace(s,"")

In [23]:
df = df.drop(columns=['images', 'tags'])

In [24]:
df.to_csv('scrape.csv', index=False)

In [25]:
import json

# Open and read the JSON file
with open('pocket_patron/scrape2.json', 'r', encoding='utf-8') as file:
    data = json.load(file)


In [30]:
data[500].get("images")[0].get("path")[:-4]

'full/2002_Hairspray'

In [38]:
from PIL import Image, ImageDraw, ImageFont
def convert_image(image_path, theater):
    image = Image.open(f"pocket_patron/downloads/{image_path}")
    image2 = Image.open("pocket_patron/downloads/full/PocketPatron.png")
    # image = image.crop((100, 100, 400, 400))
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("arial.ttf", 20)
    W, H = image.size
    draw.rectangle((0, 0, W, 233), fill=(250, 204, 21))
    _, _, w, h = draw.textbbox((0, 0), theater, font=font)
    draw.text(((648-w)/2, 170), theater, font=font, fill=(0, 0, 0), align="center")
    # draw.text((30, 10), "IMPERIAL THEATER", font=font, fill=(0, 0, 0), align="center")
    Image.Image.paste(image, image2, (45, 40))
    image.save(f"new/{image_path[:-4]}.png", "PNG")

for show in data:
        image_path = show["images"][0]["path"]
        theater = show["theater"].upper().replace("", " ").strip()

        convert_image(image_path, theater)


In [45]:
image_path  = "full/1995_Fool_Moon.jpg"
theater = "Gerard Schoenfeld Theatre"

image = Image.open(f"pocket_patron/downloads/{image_path}")
image2 = Image.open("pocket_patron/downloads/full/PocketPatron.png")
# image = image.crop((100, 100, 400, 400))
draw = ImageDraw.Draw(image)
font = ImageFont.truetype("arial.ttf", 20)
W, H = image.size
print(W, H)
draw.rectangle((0, 0, W, H*(233/1024)), fill=(250, 204, 21))
_, _, w, h = draw.textbbox((0, 0), theater, font=font)
draw.text(((648-w)/2, 170), theater, font=font, fill=(0, 0, 0), align="center")
Image.Image.paste(image, image2, (45, 40))
image.show()

257 400


In [43]:
233/1024

0.2275390625

In [50]:
"Gerwshwin Theater Circle in the Square".upper().replace("", " ").strip()

'G E R W S H W I N   T H E A T E R   C I R C L E   I N   T H E   S Q U A R E'

In [1]:
import random
import math

# List of Broadway shows
shows = [
    "Dear Evan Hansen", "Hadestown", "Phantom of the Opera", "Annie",
    "Ain't Too Proud", "Mean Girls", "Secret Garden", "Hairspray", "Six",
    "Jersey Boys", "Heathers", "Les Miserables", "Moulin Rouge", "Tina",
    "Mamma Mia", "MJ: The Musical", "POTUS", "Chicago", "The Book of Mormon",
    "The Notebook", "Sweeney Todd", "Funny Girl", "Come From Away", "Newsies",
    "Wicked", "Company", "Kimberly Akimbo", "Waitress", "Cabaret",
    "Fiddler on the Roof", "The Outsiders", "Back to the Future",
    "La Cage aux Folles", "Waiting for Godot"
]

# Initialize Elo scores
def initialize_scores(show_list):
    return {show: 1000 for show in show_list}

scores = initialize_scores(shows)
K = 32  # Sensitivity factor for Elo updates

# Track compared pairs to avoid duplicates
compared_pairs = set()

# Elo update function
def elo_update(winner, loser, scores):
    R_w = scores[winner]
    R_l = scores[loser]
    
    E_w = 1 / (1 + 10 ** ((R_l - R_w) / 400))
    E_l = 1 - E_w
    
    scores[winner] += K * (1 - E_w)
    scores[loser] += K * (0 - E_l)

# Select optimal pair for comparison based on score difference
def optimal_pair_selection(scores):
    sorted_shows = sorted(scores.items(), key=lambda x: x[1])
    optimal_pairs = []
    for i in range(len(sorted_shows)):
        for j in range(i + 1, len(sorted_shows)):
            pair = (sorted_shows[i][0], sorted_shows[j][0])
            if pair not in compared_pairs and (pair[1], pair[0]) not in compared_pairs:
                diff = abs(sorted_shows[i][1] - sorted_shows[j][1])
                optimal_pairs.append((pair[0], pair[1], diff))
    optimal_pairs.sort(key=lambda x: x[2])  # Sort by smallest score difference
    return optimal_pairs[:10]  # Return top 10 most optimal pairs

# Pairwise comparison function
def compare_shows(scores):
    pairs = optimal_pair_selection(scores)
    if not pairs:
        print("No more unique pairs to compare.")
        return False
    pair = random.choice(pairs)
    show1, show2 = pair[0], pair[1]
    compared_pairs.add((show1, show2))
    print(f"Which show do you prefer?")
    print(f"1: {show1}")
    print(f"2: {show2}")
    print("Type 'exit' to finish comparisons early.")
    choice = input("Enter 1, 2, or 'exit': ").strip()
    
    if choice == '1':
        elo_update(show1, show2, scores)
    elif choice == '2':
        elo_update(show2, show1, scores)
    elif choice.lower() == 'exit':
        return False
    else:
        print("Invalid choice. Try again.")
    return True

# Normalize scores to a 0–10 scale
def normalize_scores(scores):
    min_rating = min(scores.values())
    max_rating = max(scores.values())
    return {show: 10 * (score - min_rating) / (max_rating - min_rating) for show, score in scores.items()}

# Display rankings
def display_rankings(scores):
    normalized = normalize_scores(scores)
    ranked = sorted(normalized.items(), key=lambda x: x[1], reverse=True)
    print("\nBroadway Show Rankings:")
    for rank, (show, score) in enumerate(ranked, start=1):
        print(f"{rank}. {show} - Score: {score:.2f}")

# Calculate optimal number of comparisons
def calculate_optimal_comparisons(num_shows):
    total_pairs = (num_shows * (num_shows - 1)) / 2
    optimal_comparisons = math.ceil(0.25 * total_pairs)
    print(f"Optimal number of comparisons for {num_shows} shows: {optimal_comparisons}")
    return optimal_comparisons

# Interactive comparison loop
def run_comparison_loop(num_comparisons):
    for _ in range(num_comparisons):
        if not compare_shows(scores):
            break
    display_rankings(scores)

# Example usage
# Run this in your notebook:
# num_shows_seen = len(shows)
# optimal_comparisons = calculate_optimal_comparisons(num_shows_seen)
# run_comparison_loop(optimal_comparisons)


In [2]:
num_shows_seen = len(shows)
optimal_comparisons = calculate_optimal_comparisons(num_shows_seen)

Optimal number of comparisons for 34 shows: 141
